# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
\begin{equation}
A_x = \frac{I(i+1,j) - I(i-1,j)}{2}
\end{equation}
\begin{equation}
A_y = \frac{I(i,j+1) - I(i,j-1)}{2}
\end{equation}
\begin{equation}
A_{xy} = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}
\end{equation}

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/'

fileName = "ainvert.py"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)


In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import ainvert

def calculate_derivatives(img, i, j):
    h, w = img.shape

    i_m1 = max(0, i-1)
    i_p1 = min(w-1, i+1)
    j_m1 = max(0, j-1)
    j_p1 = min(h-1, j+1)

    fx = (img[j, i_p1] - img[j, i_m1]) / 2
    fy = (img[j_p1, i] - img[j_m1, i]) / 2
    fxy = (img[j_p1, i_p1] - img[j_m1, i] - img[j, i_m1] + img[j, i]) / 4

    return fx, fy, fxy

In [ ]:
def bicubic_interpolate(img, scale_factor):
    h, w = img.shape
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    output = np.zeros((new_h, new_w))

    for y in range(new_h):
        for x in range(new_w):
            src_x = x / scale_factor
            src_y = y / scale_factor

            x0 = int(src_x)
            y0 = int(src_y)

            if x0 >= w-1 or y0 >= h-1:
                output[y, x] = img[min(h-1, y0), min(w-1, x0)]
                continue

            dx = src_x - x0
            dy = src_y - y0

            A = img[y0, x0]
            B = img[y0, x0+1]
            C = img[y0+1, x0+1]
            D = img[y0+1, x0]

            Ax, Ay, Axy = calculate_derivatives(img, x0, y0)
            Bx, By, Bxy = calculate_derivatives(img, x0+1, y0)
            Cx, Cy, Cxy = calculate_derivatives(img, x0+1, y0+1)
            Dx, Dy, Dxy = calculate_derivatives(img, x0, y0+1)

            X = np.array([A, B, D, C, Ax, Bx, Dx, Cx, Ay, By, Dy, Cy, Axy, Bxy, Dxy, Cxy])
            alpha = ainvert.A_INVERT.dot(X)

            output[y, x] = sum(
                alpha[i + 4*j] * (dx**i) * (dy**j)
                for i in range(4)
                for j in range(4)
            )

    return np.clip(output, 0, 255).astype(np.uint8)



In [ ]:
def bilinear_interpolate(img, scale_factor):
    h, w = img.shape
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    output = np.zeros((new_h, new_w))

    for y in range(new_h):
        for x in range(new_w):
            src_x = x / scale_factor
            src_y = y / scale_factor

            x0 = int(src_x)
            y0 = int(src_y)

            if x0 >= w-1 or y0 >= h-1:
                output[y, x] = img[min(h-1, y0), min(w-1, x0)]
                continue

            dx = src_x - x0
            dy = src_y - y0

            p1 = img[y0, x0]
            p2 = img[y0, x0+1]
            p3 = img[y0+1, x0]
            p4 = img[y0+1, x0+1]

            output[y, x] = (1-dx)*(1-dy)*p1 + dx*(1-dy)*p2 + (1-dx)*dy*p3 + dx*dy*p4

    return np.clip(output, 0, 255).astype(np.uint8)


In [ ]:
def compare_interpolations(img, scale_factor=2):
    bilinear = bilinear_interpolate(img, scale_factor)
    bicubic = bicubic_interpolate(img, scale_factor)

    plt.figure(figsize=(15, 5))
    plt.subplot(131)
    plt.imshow(img, cmap='gray')
    plt.title('Original')
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(bilinear, cmap='gray')
    plt.title('Bilinear')
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(bicubic, cmap='gray')
    plt.title('Bicubic')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    print("\nOperation Count Comparison:")
    print("Bilinear Interpolation:")
    print("- Arithmetic operations per pixel: 7 (4 multiplications + 3 additions)")
    print("- Memory accesses per pixel: 4")
    print("\nBicubic Interpolation:")
    print("- Arithmetic operations per pixel: ~256 (16x16 matrix multiplication + derivatives)")
    print("- Memory accesses per pixel: 16 (for points) + additional for derivatives")

In [ ]:
img = cv2.imread('lena.png', cv2.IMREAD_GRAYSCALE)
compare_interpolations(img, scale_factor=2)